In [5]:
import pandas as pd
import time
import os
import json
from google import genai
from google.genai import types
from dotenv import load_dotenv

In [3]:
load_dotenv()
client = genai.Client(api_key=os.getenv("api_key"))

In [44]:
df = pd.read_excel("source data/database cbo.xlsx", sheet_name = 0)

In [ ]:
for idx, row in df.iterrows():
    linha_atual = idx + 1
    print(f"🔎 Processando linha {linha_atual} de {len(df)}")

    cbo = row["cbo"]
    group = row["task_group"]
    task = row["task"]
    tarefa_texto = f"{group}: {task}"
    description = row["description"]
    key = row["key"]

    prompt_usuario = f"""
Você é um especialista em análise ocupacional e economia do trabalho, com profundo conhecimento da Classificação Brasileira de Ocupações (CBO) e do impacto de tecnologias emergentes no mundo do trabalho.

Sua tarefa é avaliar a seguinte tarefa: {tarefa_texto} de id {key}, pertencente à ocupação {description}, com base em um framework que considera três tecnologias específicas: IA generativa (como Large Language Models), Visão Computacional e Robótica para ações físicas.

Com base nisso, gere os três indicadores:

Taxa de Substituição (TR): avalie o potencial de substituição da tarefa {tarefa_texto} da ocupação {description} por tecnologias de IA (IA generativa, visão computacional ou robótica) em uma escala de 1 a 5, onde 1 representa tarefas quase impossíveis de substituir por IA (exigindo habilidades humanas não sintetizáveis, como empatia genuína ou julgamento ético complexo), e 5 indica tarefas totalmente substituíveis (como processamento de dados estruturados ou ações repetitivas), facilmente substituíveis com as tecnologias atuais. Inclua uma justificativa breve para o valor atribuído.

Exemplo TR para 'Atendimento ao cliente padrão':
4 - LLMs podem resolver 80% das consultas rotineiras, mas falham em conflitos emocionais complexos
---

Taxa de Exposição (TE): avalie em uma escala de 1 a 5 o grau em que a tarefa {tarefa_texto} da ocupação {description} está exposta ou mediada pelas tecnologias mencionadas, considerando o quanto sua execução atual ou potencial pode depender, ser assistida ou aprimorada por essas ferramentas. Inclua uma justificativa breve para o valor atribuído.

Exemplo TE para 'Atendimento ao cliente padrão':
5 - A tarefa está altamente exposta a tecnologias de IA generativa, sendo frequentemente mediada por chatbots e assistentes virtuais

---

Dummy de Automação Total: indique com 1 se a tarefa {tarefa_texto} da ocupação {description} pode ser totalmente automatizada pelas tecnologias avaliadas, sem necessidade de supervisão humana significativa; e com 0 se ainda requer intervenção humana por limitações tecnológicas ou pela natureza da tarefa. Inclua uma justificativa breve para a decisão.

Exemplo Dummy para 'Atendimento ao cliente padrão':
1 - A automação total é possível para a maioria das interações padrão, sem necessidade significativa de supervisão humana, pois casos complexos são exceções

---
### *FORMATO DA RESPOSTA*

Gere um JSON com a análise para cada uma das tarefas fornecidas. A resposta deve ser *apenas o objeto JSON*, sem nenhum texto introdutório ou explicações adicionais.

[
  {{
    "key": id_tarefa #somente o id,
    "TE": <numero_de_1_a_5>,
    "justificativa_TE": "<texto_breve_e_objetivo>",
    "TR": <numero_de_1_a_5>,
    "justificativa_TR": "<texto_breve_e_objetivo>",
    "dummy": <0_ou_1>
  }}
]
"""

    try:
        tempo_inicio = time.time()

        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt_usuario,
        )
        resposta = response.text
        resposta = resposta.replace("```json", "").replace("```", "")
        tempo_fim = time.time()
        duracao = tempo_fim - tempo_inicio
        print(f"{key} processado em {duracao:.2f} segundos")

        arquivo_saida = os.path.join("json/te_google3", f"{key}.json")
        with open(arquivo_saida, "w", encoding="utf-8") as f:
            json.dump(json.loads(resposta), f, ensure_ascii=False, indent=4)

    except Exception as e:
        print(f"Erro ao processar {key} - {linha_atual}: {e}")

In [6]:
all_json = []
arquivos_com_erro = []

pasta = "json/te_tr"

for nome_arquivo in os.listdir(pasta):
    if nome_arquivo.endswith(".json"):
        caminho_arquivo = os.path.join(pasta, nome_arquivo)
        try:
            with open(caminho_arquivo, "r", encoding="utf-8") as f:
                conteudo = f.read().strip()
                if not conteudo:
                    raise ValueError("Arquivo vazio")
                dados = json.loads(conteudo)
                if isinstance(dados, list):
                    all_json.extend(dados)
                else:
                    all_json.append(dados)
        except Exception as e:
            print(f"Erro no arquivo '{nome_arquivo}': {e}")
            arquivos_com_erro.append(caminho_arquivo)


for caminho in arquivos_com_erro:
    try:
        os.remove(caminho)
        print(f"Arquivo deletado: {os.path.basename(caminho)}")
    except Exception as e:
        print(f"Erro ao deletar {os.path.basename(caminho)}: {e}")


with open("json/te_tr.json", "w", encoding="utf-8") as f:
    json.dump(all_json, f, ensure_ascii=False, indent=2)

In [7]:
with open("json/te_tr.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)

df.to_excel("results/te_tr.xlsx", index=False)